In [123]:
import time
from curl_cffi import requests as cureq
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import re

In [124]:
headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"}

In [125]:
url = f"https://www.immoweb.be/nl/search-results/huis/te-koop?countries=BE&page=1&orderBy=relevance"

resp = cureq.get(url, headers=headers, impersonate="chrome")

print(resp.status_code)

data = resp.json()

data

total_houses_1_page = int(data['range'].split('-')[1])

total_number_of_houses= data['totalItems']

number_of_pages = total_number_of_houses//total_houses_1_page 

print(f'Immoweb contains {total_number_of_houses} listings on {number_of_pages} pages')


200
Immoweb contains 9969 listings on 343 pages


In [126]:
data

{'criteria': {'countries': 'BE',
  'propertyTypes': 'HOUSE',
  'transactionTypes': 'FOR_SALE',
  'page': '1'},
 'sort': 'relevance',
 'results': [{'id': 20243528,
   'cluster': {'minPrice': None,
    'maxPrice': None,
    'minRoom': None,
    'maxRoom': None,
    'minSurface': None,
    'maxSurface': None,
    'projectInfo': None,
    'bedroomRange': '',
    'surfaceRange': ''},
   'customerLogoUrl': 'https://media.immowebstatic.be/customers/0012p00002TxtsKAAR/logo/4af7f7124dca6f1bd930e53bbf60ab32.jpg',
   'customerName': 'Multimmo bvba',
   'flags': {'main': 'new', 'secondary': [], 'percentSold': None},
   'media': {'pictures': [{'smallUrl': 'https://media-resize.immowebstatic.be/classifieds/fde868d3-4c7d-40e0-9178-ad1589a2bd0a/100x100/2b9f9a01271b4725f1a014ab0e0736c5.jpg',
      'mediumUrl': 'https://media-resize.immowebstatic.be/classifieds/fde868d3-4c7d-40e0-9178-ad1589a2bd0a/300x300/2b9f9a01271b4725f1a014ab0e0736c5.jpg',
      'largeUrl': 'https://media-resize.immowebstatic.be/cla

In [127]:
#FIRST SCRAPE

def collect_data(pages: int, headers: dict):
    """
    This function collects data from the specified number of pages and returns a list.

    Args:
    pages (int): Number of pages to collect data from.
    headers (dict): HTTP headers to include in the requests.
    """
    if not headers:
        raise ValueError("Headers are required to make the request.")

    data_collection = []

    # Start the timer
    start_time = time.perf_counter()
    print(f"Start scraping {pages} pages...")

    # Loop through the pages and collect data
    for page in range(1, pages + 1):
        url = f"https://www.immoweb.be/nl/search-results/huis/te-koop?countries=BE&page={page}&orderBy=relevance"
        
        try:
            # Send the request
            print(f"Scraping page {page}...")
            resp = cureq.get(url, headers=headers, impersonate="chrome")
            resp.raise_for_status()  # Raise an error for bad responses (4xx or 5xx)

            # Try to parse the JSON response
            data = resp.json()
            data_collection.append(data)
            print(f"Page {page} scraped successfully.")
        
        except requests.exceptions.RequestException as req_err:
            # Handle network-related errors
            print(f"Network error on page {page}: {req_err}")
        except ValueError:
            # Handle JSON decoding errors
            print(f"Error decoding JSON on page {page}.")
        except Exception as e:
            # Catch any other exceptions
            print(f"An error occurred on page {page}: {e}")

    # Stop the timer and print the elapsed time
    end_time = time.perf_counter()
    elapsed_time = end_time - start_time
    print(f"Data scraping completed in {elapsed_time:.2f} seconds.")

    return data_collection


In [128]:
def parse_data(listings: list):
    """
    Parses the listings data to extract relevant property details.

    Args:
    listings (list): List of raw listings data.

    Returns:
    all_properties (list): List of dictionaries with parsed property data.
    """
    all_properties = []

    for listing in listings:
        # Get the results list (default to an empty list if not found)
        results_list = listing.get('results', [])

        for result in results_list:
            # Extract property details using nested get() for safety
            property_details = result.get('property', {})
            location = property_details.get('location', {})
            transaction = result.get('transaction', {}).get('sale', {})

            property_data = {
                'id': result.get('id'),
                'type': property_details.get('type'),
                'subtype': property_details.get('subtype'),
                'country': location.get('country'),
                'region': location.get('region'),
                'locality_name': location.get('locality'),
                'locality_code': location.get('postalCode'),
                'bedroom_count': property_details.get('bedroomCount', "None"),
                'net_habitable_surface': property_details.get('netHabitableSurface'),
                'land_surface': property_details.get('landSurface'),
                'room_count': property_details.get('roomCount', "None"),
                'transaction_type': result.get('transaction', {}).get('type'),
                'sale_annuity': transaction.get('lifeAnnuity'),
                'price': transaction.get('price'),
                'old_price': transaction.get('oldPrice')
            }

            all_properties.append(property_data)

    print(f"Extracted {len(all_properties)} properties from data.")
    return all_properties


In [129]:
#Testing the first scrape

listings = collect_data(2, headers)
all_properties = parse_data(listings)

Start scraping 2 pages...
Scraping page 1...
Page 1 scraped successfully.
Scraping page 2...
Page 2 scraped successfully.
Data scraping completed in 0.77 seconds.
Extracted 60 properties from data.


In [130]:
all_properties

[{'id': 20243528,
  'type': 'HOUSE',
  'subtype': 'HOUSE',
  'country': 'België',
  'region': 'Vlaanderen',
  'locality_name': 'BEERSEL',
  'locality_code': '1650',
  'bedroom_count': 2,
  'net_habitable_surface': 110,
  'land_surface': 5390,
  'room_count': None,
  'transaction_type': 'FOR_SALE',
  'sale_annuity': None,
  'price': 499000,
  'old_price': None},
 {'id': 20243985,
  'type': 'HOUSE_GROUP',
  'subtype': 'HOUSE_GROUP',
  'country': 'België',
  'region': 'Vlaanderen',
  'locality_name': 'Deinze Astene',
  'locality_code': '9800',
  'bedroom_count': None,
  'net_habitable_surface': None,
  'land_surface': None,
  'room_count': None,
  'transaction_type': 'FOR_SALE',
  'sale_annuity': None,
  'price': None,
  'old_price': None},
 {'id': 20221782,
  'type': 'HOUSE',
  'subtype': 'HOUSE',
  'country': 'België',
  'region': 'Vlaanderen',
  'locality_name': 'DEURNE',
  'locality_code': '2100',
  'bedroom_count': 3,
  'net_habitable_surface': 89,
  'land_surface': 233,
  'room_coun

In [131]:
#create a first list with results in dic

listings_1 = []  

for house in all_properties:
    listing_1 = {}  
    listing_1['id'] = house.get('id')
    listing_1['locality_name'] = house.get('locality_name')
    listing_1['Postal_code'] = house.get('locality_code')
    listing_1['Price'] = house.get('price')
    listing_1['Type_of_property'] = house.get('subtype')
    listing_1['Number_of_rooms'] = house.get('room_count')
    listing_1['Living_area'] = house.get('net_habitable_surface')
    if house.get('sale_annuity') is None:
        listing_1['Type_of_sale'] = house.get('transaction_type')
    
    listings_1.append(listing_1)  

listings_1

    
    

[{'id': 20243528,
  'locality_name': 'BEERSEL',
  'Postal_code': '1650',
  'Price': 499000,
  'Type_of_property': 'HOUSE',
  'Number_of_rooms': None,
  'Living_area': 110,
  'Type_of_sale': 'FOR_SALE'},
 {'id': 20243985,
  'locality_name': 'Deinze Astene',
  'Postal_code': '9800',
  'Price': None,
  'Type_of_property': 'HOUSE_GROUP',
  'Number_of_rooms': None,
  'Living_area': None,
  'Type_of_sale': 'FOR_SALE'},
 {'id': 20221782,
  'locality_name': 'DEURNE',
  'Postal_code': '2100',
  'Price': 284000,
  'Type_of_property': 'HOUSE',
  'Number_of_rooms': None,
  'Living_area': 89,
  'Type_of_sale': 'FOR_SALE'},
 {'id': 20236558,
  'locality_name': 'ANTWERPEN',
  'Postal_code': '2018',
  'Price': 389900,
  'Type_of_property': 'HOUSE',
  'Number_of_rooms': None,
  'Living_area': 150,
  'Type_of_sale': 'FOR_SALE'},
 {'id': 20236560,
  'locality_name': 'BORGERHOUT',
  'Postal_code': '2140',
  'Price': 389900,
  'Type_of_property': 'HOUSE',
  'Number_of_rooms': None,
  'Living_area': 150,
  

In [132]:
# Convert to DataFrame and export to CSV
dataframe_first_scrape = pd.DataFrame(listings_1)
csv_path = r"C:\Users\Rik\Desktop\immoeliza\scraper\immo_scraper_1.csv"
dataframe_first_scrape.to_csv(csv_path, index=False)

print(f"Data from first scrape saved to {csv_path}")

Data from first scrape saved to C:\Users\Rik\Desktop\immoeliza\scraper\immo_scraper_1.csv


In [133]:
## SECOND SCRAPE

print("Start second scraping")
    
data_collection_2 = []

for property in listings_1:
    
    base_url = "https://www.immoweb.be/nl/zoekertje/"
    property_url = base_url + str(property['id'])
    
    try:
        # Send request to property URL
        r = requests.get(property_url, headers=headers)
        r.raise_for_status()  
        
        # Parse the page content
        soup = BeautifulSoup(r.content, "html.parser")
        print(f"Page {property_url} scraped successfully.")
        
        # Try to extract the specific script tag
        script_tag = soup.select_one('div.classified script[type="text/javascript"]')
        
        if script_tag:
            # Process the script_tag here
            print("Script tag found and processed.")
        else:
            print(f"No script tag found on {property_url}")
    
    except Exception as e:
        # Print the error message and continue to the next property
        print(f"Error on page {property_url}: {e}")
        continue
    
    # Extract the JavaScript object
    js_content = script_tag.string
    
    # Find the start and end of the JSON object
    start = js_content.find('{')
    end = js_content.rfind('}') + 1
    
    # Extract and parse the JSON data
    json_data = json.loads(js_content[start:end])
    
    #append json_data to the list data_collection2
    data_collection_2.append(json_data)




Start second scraping
Page https://www.immoweb.be/nl/zoekertje/20243528 scraped successfully.
Script tag found and processed.
Page https://www.immoweb.be/nl/zoekertje/20243985 scraped successfully.
Script tag found and processed.
Page https://www.immoweb.be/nl/zoekertje/20221782 scraped successfully.
Script tag found and processed.
Page https://www.immoweb.be/nl/zoekertje/20236558 scraped successfully.
Script tag found and processed.
Page https://www.immoweb.be/nl/zoekertje/20236560 scraped successfully.
Script tag found and processed.
Page https://www.immoweb.be/nl/zoekertje/20242082 scraped successfully.
Script tag found and processed.
Page https://www.immoweb.be/nl/zoekertje/20241609 scraped successfully.
Script tag found and processed.
Page https://www.immoweb.be/nl/zoekertje/20242816 scraped successfully.
Script tag found and processed.
Page https://www.immoweb.be/nl/zoekertje/20243152 scraped successfully.
Script tag found and processed.
Page https://www.immoweb.be/nl/zoekertje/2

In [134]:

with open('listings.json', 'w') as f:
    json.dump(data_collection_2[0], f, indent=4)

In [135]:
#parse en merge the data from data_collection_2 in listings_2
# listings_2 = []

# for house in data_collection_2:
#     listing_2 = {}
    
#     try:
#         if house['id']:
#             listing_2['id'] = house['id']
#         else:
#             listing_2['id'] = None
#     except KeyError:
#         listing_2['id'] = None
    
#     property_details = house.get('property', {})
    
#     if property_details.get('kitchen', {}).get('type') == "INSTALLED":
#         listing_2['Equipped_kitchen'] = True
#     else:
#         listing_2['Equipped_kitchen'] = False
        
#     if property_details.get('transaction',{}).get('sale',{}).get('isFurnished') == "null":
#         listing_2['Furnished'] = False
#     else:
#         listing_2['Furnished'] = True
    
#     if property_details.get('fireplaceExists') == "false":
#         listing_2['Open_fire'] = False
#     else:
#         listing_2['Open_fire'] = True
        
#     if property_details.get('hasTerrace') == "false":
#         listing_2['Terrace'] = 'null'
#     else:
#         listing_2['Terrace'] = property_details.get('terraceSurface')
        
#     if property_details.get('hasGarden') == "false":
#         listing_2['Garden'] = 'null'
#     else:
#         listing_2['Garden'] = property_details.get('gardenSurface')
        
#     listing_2['Number_of_facades'] = property_details.get('building', {}).get('facadeCount')
    
#     if property_details.get('hasSwimmingPool') == 'null':
#         listing_2['Swimming_Pool'] = False
#     else:
#         listing_2['Swimming_Pool'] = True
        
#     listing_2['State_of_building'] = property_details.get('building', {}).get('condition')
    
#     listings_2.append(listing_2)

listings_2 = []

for house in data_collection_2:
    listing_2 = {}
    
    
    listing_2['id'] = house.get('id', None)
    
    
    property_details = house.get('property', {})
    
    
    kitchen_details = property_details.get('kitchen')
    if isinstance(kitchen_details, dict) and kitchen_details.get('type') == "INSTALLED":
        listing_2['Equipped_kitchen'] = True
    else:
        listing_2['Equipped_kitchen'] = False
    
    
    is_furnished = property_details.get('transaction', {}).get('sale', {}).get('isFurnished')
    listing_2['Furnished'] = False if is_furnished == "null" else True
    
    
    fireplace_exists = property_details.get('fireplaceExists')
    listing_2['Open_fire'] = False if fireplace_exists == "false" else True
    
    
    has_terrace = property_details.get('hasTerrace')
    if has_terrace == "false":
        listing_2['Terrace'] = 'null'
    else:
        listing_2['Terrace'] = property_details.get('terraceSurface')
    
    
    has_garden = property_details.get('hasGarden')
    if has_garden == "false":
        listing_2['Garden'] = 'null'
    else:
        listing_2['Garden'] = property_details.get('gardenSurface')
    
    
    building_details = property_details.get('building')
    if isinstance(building_details, dict):
        listing_2['Number_of_facades'] = building_details.get('facadeCount', None)
    else:
        listing_2['Number_of_facades'] = None
    
    
    has_swimming_pool = property_details.get('hasSwimmingPool')
    listing_2['Swimming_Pool'] = False if has_swimming_pool == "null" else True
    
   
    if isinstance(building_details, dict):
        listing_2['State_of_building'] = building_details.get('condition', None)
    else:
        listing_2['State_of_building'] = None
    
    
    listings_2.append(listing_2)


In [136]:
listings_2[0]

{'id': 20243528,
 'Equipped_kitchen': True,
 'Furnished': True,
 'Open_fire': True,
 'Terrace': None,
 'Garden': 5200,
 'Number_of_facades': 4,
 'Swimming_Pool': True,
 'State_of_building': 'GOOD'}

In [137]:
dataframe_second_scrape = pd.DataFrame(listings_2)
csv_path = r"C:\Users\Rik\Desktop\immoeliza\scraper\immo_scraper_2.csv"
dataframe_second_scrape.to_csv(csv_path, index=False)

print(f"Data from second scrape saved to {csv_path}")

Data from second scrape saved to C:\Users\Rik\Desktop\immoeliza\scraper\immo_scraper_2.csv


In [138]:
# Merge all the data

merged_df = pd.merge(dataframe_first_scrape, dataframe_second_scrape, on='id', how='inner')

print(merged_df)
csv_path = r"C:\Users\Rik\Desktop\immoeliza\scraper\immo_scraper_merged.csv"
merged_df.to_csv(csv_path, index=False)

print(f"Merged dat saved to {csv_path}")


          id        locality_name Postal_code     Price Type_of_property  \
0   20243528              BEERSEL        1650  499000.0            HOUSE   
1   20243985        Deinze Astene        9800       NaN      HOUSE_GROUP   
2   20221782               DEURNE        2100  284000.0            HOUSE   
3   20236558            ANTWERPEN        2018  389900.0            HOUSE   
4   20236560           BORGERHOUT        2140  389900.0            HOUSE   
5   20242082                DWORP        1653  359000.0            HOUSE   
6   20241609               EDEGEM        2650  384950.0            HOUSE   
7   20242816             Wevelgem        8560       NaN      HOUSE_GROUP   
8   20243152               Gavere        9890       NaN      HOUSE_GROUP   
9   20242879         Lichtervelde        8810       NaN      HOUSE_GROUP   
10  20242880               Aalter        9880       NaN      HOUSE_GROUP   
11  20240478               Lierde        9570       NaN      HOUSE_GROUP   
12  20240433

In [140]:
merged_df.head(15)

,id,locality_name,Postal_code,Price,Type_of_property,Number_of_rooms,Living_area,Type_of_sale,Equipped_kitchen,Furnished,Open_fire,Terrace,Garden,Number_of_facades,Swimming_Pool,State_of_building
0,20243528,BEERSEL,1650,499000.0,HOUSE,NaN,110.0,FOR_SALE,True,True,True,NaN,5200.0,4.0,True,GOOD
1,20243985,Deinze Astene,9800,NaN,HOUSE_GROUP,NaN,NaN,FOR_SALE,False,True,True,NaN,NaN,NaN,True,None
2,20221782,DEURNE,2100,284000.0,HOUSE,NaN,89.0,FOR_SALE,False,True,True,3.0,97.0,2.0,True,TO_BE_DONE_UP
3,20236558,ANTWERPEN,2018,389900.0,HOUSE,NaN,150.0,FOR_SALE,False,True,True,5.0,19.0,2.0,True,JUST_RENOVATED
4,20236560,BORGERHOUT,2140,389900.0,HOUSE,NaN,150.0,FOR_SALE,False,True,True,5.0,19.0,2.0,True,JUST_RENOVATED
5,20242082,DWORP,1653,359000.0,HOUSE,NaN,78.0,FOR_SALE,True,True,True,NaN,NaN,4.0,True,GOOD
6,20241609,EDEGEM,2650,384950.0,HOUSE,NaN,160.0,FOR_SALE,False,True,True,NaN,70.0,2.0,True,TO_BE_DONE_UP
7,20242816,Wevelgem,8560,NaN,HOUSE_GROUP,NaN,NaN,FOR_SALE,False,True,True,NaN,NaN,NaN,True,None
8,20243152,Gavere,9890,NaN,HOUSE_GROUP,NaN,NaN,FOR_SALE,False,True,True,NaN,NaN,NaN,True,None
9,20242879,Lichtervelde,8810,NaN,HOUSE_GROUP,NaN,NaN,FOR_SALE,False,True,True,NaN,NaN,NaN,True,None
